In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import numpy as np
from itertools import zip_longest

In [2]:
def bs4_soup(pages):
    URL = f'https://www.flipkart.com/search?q=infinix+mobile&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={pages}'
    request = requests.get(URL)
    soup = bs(request.text,'html.parser')
    soup = soup.find('div',class_ = '_1YokD2 _3Mn1Gg')

    return soup , URL

In [3]:
def mob_specs(soup):
    names = soup.find_all('div', class_='_4rR01T')
    mob_names = [i.text if i else None for i in names]

    ratings = soup.find_all('div', class_='_3LWZlK')
    mob_ratings = [i.text if i else None for i in ratings]

    price = soup.find_all('div', class_='_30jeq3 _1_WHN1')
    mob_price = [i.text if i else None for i in price]

    img = soup.find_all('div', class_='CXW8mj')
    mob_img_URL = [i.find('img')['src'] if i.find('img') else None for i in img]

    return mob_names, mob_ratings, mob_price, mob_img_URL


In [4]:
def features_from_mob_links(soup):
    mobile_links = ['https://www.flipkart.com' + i['href'] for i in soup.find_all('a', class_='_1fQZEK')]
    mobile_req = [requests.get(url=i) for i in mobile_links]
    mobile_soup = [bs(i.text, 'html.parser').find('div', class_='_1YokD2 _2GoDe3') for i in mobile_req]
    mobile_specs_soup = [i.find_all('div', class_='_3k-BhJ') for i in mobile_soup]

    storage_ram = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Memory' in str(specs_.text) and 'Storage' in str(specs_.text)]
    storage_ram_cross_checked = [i for i in storage_ram if 'Internal' in i.split()]

    os_processor = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Os' in str(specs_.text) and 'Processor' in str(specs_.text)]
    os_processor_cross_checked = [i for i in os_processor if 'Operating' in i.split()]

    camera = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Camera' in str(specs_.text) and 'FeaturesPrimary' in str(specs_.text)]
    camera_cross_checked = [i for i in camera if 'Primary' and 'Camera' in i.split()]

    display = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Display' in str(specs_.text) and 'FeaturesDisplay' in str(specs_.text)]
    display_cross_checked = [i for i in display if 'Display' and 'cm' in i.split()]

    network = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Connectivity' in str(specs_.text) and 'FeaturesNetwork' in str(specs_.text)]
    network_cross_checked = [i for i in network if 'Network' in i.split()]

    battery = [specs_.find('table', class_='_14cfVK').text if specs_ else None for mob_specs in mobile_specs_soup for specs_ in mob_specs if 'Battery' in str(specs_.text) and 'FeaturesBattery' in str(specs_.text)]
    battery_cross_checked = [i for i in battery if 'Battery' in i.split()]

    return mobile_specs_soup, storage_ram_cross_checked, os_processor_cross_checked, camera_cross_checked, display_cross_checked, network_cross_checked, battery_cross_checked


In [5]:
def fill_nan_values_func(mob_names, mob_ratings, mob_price, mob_img_URL, storage_ram, os_processor, camera, display, network, battery):
    tuple_data = zip_longest(mob_names, mob_ratings, mob_price, mob_img_URL, storage_ram, os_processor, camera, display, network, battery,fillvalue=np.nan)
    
    data = [i for i in tuple_data]
    return data
    

In [6]:
def print_len(mob_names, mob_ratings, mob_price, mob_img_URL, storage_ram, os_processor, camera, display, network, battery):
    print(f"Length of mob_names: {len(mob_names)}")
    print(f"Length of mob_ratings: {len(mob_ratings)}")
    print(f"Length of mob_price: {len(mob_price)}")
    print(f"Length of mob_img_URL: {len(mob_img_URL)}")
    print(f"Length of storage_ram: {len(storage_ram)}")
    print(f"Length of os_processor: {len(os_processor)}")
    print(f"Length of camera: {len(camera)}")
    print(f"Length of display: {len(display)}")
    print(f"Length of network: {len(network)}")
    print(f"Length of battery: {len(battery)}")   

In [7]:
df = pd.DataFrame()

page_start = 1
page_ends = 9

for pages in range(page_start, page_ends):

    soup, URL = bs4_soup(pages=pages)
    
    mob_names, mob_ratings, mob_price, mob_img_URL = mob_specs(soup)
    mobile_specs_soup, storage_ram, os_processor, camera, display, network, battery = features_from_mob_links(soup=soup)
    
    data = fill_nan_values_func(mob_names, mob_ratings, mob_price, mob_img_URL, storage_ram, os_processor, camera, display, network, battery)

    if mobile_specs_soup is not None and soup is not None:
        
        print_len(mob_names, mob_ratings, mob_price, mob_img_URL, storage_ram, os_processor, camera, display, network, battery)

        page_df = pd.DataFrame(data=data,columns=['name', 'ratings', 'price', 'imgURL', 'storage_ram', 'camera',
       'oS_Processor', 'display', 'network', 'battery'])

        if not page_df.empty:
            df = pd.concat(objs=[df, page_df], ignore_index=True)
            print(f'scraping data from page:{pages}')
            print(f'working in {URL} Page')
            print(f'page df : page_shape : {page_df.shape} \n {page_df.sample(2)}')
            print(f'df : df shape : {df.shape} \n {df.sample(2)}')
            print()

    else:
        print(f"Skipping page {pages} due to empty mobile_specs_soup or soup.")

Length of mob_names: 24
Length of mob_ratings: 24
Length of mob_price: 24
Length of mob_img_URL: 24
Length of storage_ram: 24
Length of os_processor: 24
Length of camera: 24
Length of display: 24
Length of network: 24
Length of battery: 24
scraping data from page:1
working in https://www.flipkart.com/search?q=infinix+mobile&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1 Page
page df : page_shape : (24, 10) 
                                       name ratings   price   
11    Infinix SMART 7 (Night Black, 64 GB)     4.3  ₹7,299  \
4   Infinix HOT 30i (Diamond White, 64 GB)     4.3  ₹7,999   

                                               imgURL   
11  https://rukminim2.flixcart.com/image/312/312/x...  \
4   https://rukminim2.flixcart.com/image/312/312/x...   

                                          storage_ram   
11  Internal Storage64 GBRAM4 GBExpandable Storage...  \
4   Internal Storage64 GBRAM4 GBExpandable Storage...   

                         

In [8]:
df

,name,ratings,price,imgURL,storage_ram,camera,oS_Processor,display,network,battery
0,"Infinix SMART 7 (Azure Blue, 64 GB)",4.3,"₹7,299",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,Operating SystemAndroid 12Processor TypeUnisoc...,Primary Camera AvailableYesPrimary Camera13MP ...,Display Size16.76 cm (6.6 inch)Resolution1612 ...,"Network Type4G, 3G, 2GSupported Networks4G LTE...",Battery Capacity6000 mAhBattery TypeLithium-io...
1,"Infinix HOT 30i (Marigold, 128 GB)",4.3,"₹8,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBExpandable Storag...,Operating SystemAndroid 12Processor TypeG37Pro...,Primary Camera AvailableYesPrimary Camera50MP ...,Display Size16.76 cm (6.6 inch)Resolution1612 ...,"Network Type2G, 3G, 4GSupported Networks4G LTE...",Battery Capacity5000 mAhBattery TypeLithium-io...
2,"Infinix HOT 30i (Marigold, 64 GB)",4.3,"₹7,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,Operating SystemAndroid 12Processor TypeG37Pro...,Primary Camera AvailableYesPrimary Camera50MP ...,Display Size16.76 cm (6.6 inch)Resolution1612 ...,"Network Type2G, 3G, 4GSupported Networks4G LTE...",Battery Capacity5000 mAhBattery TypeLithium-io...
3,"Infinix HOT 30i (Diamond White, 128 GB)",4.3,"₹8,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBExpandable Storag...,Operating SystemAndroid 12Processor TypeG37Pro...,Primary Camera AvailableYesPrimary Camera50MP ...,Display Size16.76 cm (6.6 inch)Resolution1612 ...,"Network Type2G, 3G, 4GSupported Networks4G LTE...",Battery Capacity5000 mAhBattery TypeLithium-io...
4,"Infinix HOT 30i (Diamond White, 64 GB)",4.3,"₹7,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,Operating SystemAndroid 12Processor TypeG37Pro...,Primary Camera AvailableYesPrimary Camera50MP ...,Display Size16.76 cm (6.6 inch)Resolution1612 ...,"Network Type2G, 3G, 4GSupported Networks4G LTE...",Battery Capacity5000 mAhBattery TypeLithium-io...
...,...,...,...,...,...,...,...,...,...,...
187,"Infinix Hot S3X (Milan Black, 32 GB)",4.2,"₹10,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage32 GBRAM3 GBExpandable Storage...,Operating SystemAndroid Oreo 8.1Processor Type...,Primary Camera AvailableYesPrimary Camera13MP ...,Display Size15.75 cm (6.2 inch)Resolution1500 ...,"Network Type3G, 4G, 2GSupported NetworksGSM, W...",Battery Capacity4000 mAh
188,"Infinix Note 10 Pro (Nordic Secret, 256 GB)",4.2,"₹16,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage256 GBRAM8 GBExpandable Storag...,Operating SystemAndroid 11Processor TypeMediaT...,Primary Camera AvailableYesPrimary Camera64MP ...,Display Size17.65 cm (6.95 inch)Resolution2460...,"Network Type4G, 3G, 2GSupported Networks4G LTE...",Battery Capacity5000 mAh
189,"Infinix Note 5 Stylus (Bordeaux Red, 64 GB)",3.9,"₹16,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage64 GBRAM4 GBExpandable Storage...,Operating SystemAndroid Oreo 8.1Processor Type...,Primary Camera AvailableYesPrimary Camera16MP ...,Display Size15.06 cm (5.93 inch)Resolution1080...,"Network Type3G, 4G VOLTE, 4G, 2GSupported Netw...",Battery Capacity4000 mAhBattery TypeLi-ion Pol...
190,"Infinix Note 5 Stylus (Charcoal Blue, 64 GB)",3.9,"₹16,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage64 GBRAM4 GBExpandable Storage...,Operating SystemAndroid Oreo 8.1Processor Type...,Primary Camera AvailableYesPrimary Camera16MP ...,Display Size15.06 cm (5.93 inch)Resolution1080...,"Network Type3G, 4G VOLTE, 4G, 2GSupported Netw...",Battery Capacity4000 mAhBattery TypeLi-ion Pol...


In [9]:
print(os.getcwd())
os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem')
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem\notebook\data_Ingestion\scraping
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [10]:
df['imgURL'][0]

'https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/u/c/u/-original-imagnanh2yvpc4zx.jpeg?q=70'

In [11]:
df.to_csv(r'data\raw_data\infinix_mobile_data.csv',index=False)